## Data Preparing

In [1]:
import h5py
import numpy as np
import pickle

data = h5py.File('/home/sunji/ANN/glove_200_angular/glove-200-angular.hdf5', 'r')
data_train = np.array(data['train'])
data_test = np.array(data['test'])
with open('/home/sunji/ANN/glove_200_angular/clusters_glove_200_angular.pkl', 'rb') as f:
    clusters = pickle.load(f)
with open('/home/sunji/ANN/glove_200_angular/ground_truth_glove_200_angular_0_4_0_5.pkl', 'rb') as f:
    ground_truth_total = pickle.load(f)

In [2]:
ground_truth_total_level = [[[] for _ in range(10000)] for _ in range(100)]
for clus in range(100):
    for t in ground_truth_total[clus]:
        ground_truth_total_level[t[0]][t[1]].append(t)

In [3]:
centroids = []
for cluster in clusters:
    centroids.append(np.mean(cluster))

In [4]:
from numpy import dot
from numpy.linalg import norm
from scipy import spatial

def euclidean_dist_normalized(x1, x2=None, eps=1e-8):
    if np.isnan(x2):
        return 1.0
    left = x1
    right = x2
    return np.sqrt(((left - right) ** 2).mean())

def angular_dist(x1, x2=None, eps=1e-8):
    cosine_sim = 1 - spatial.distance.cosine(x1, x2)
#     print (cosine_sim)
    distance = np.arccos(cosine_sim) / 3.14159267
    return distance

test_features = []
test_thresholds = []
test_distances = []
test_targets = []
test_cards = []
slot = 0.002
for query_id in range(8000,10000):
    cardinality = [0 for _ in range(100)]
    distances2centroids = []
    for cc in centroids:
        distances2centroids.append(angular_dist(data_test[query_id], cc))
    for threshold_id, threshold in enumerate(np.arange(0.4, 0.5, slot)):
        indicator = []
        cards = []
        for cluster_id in range(100):
            cardinality[cluster_id] += ground_truth_total_level[cluster_id][query_id][threshold_id][-1]
            if cardinality[cluster_id] > 0:
                indicator.append(1)
            else:
                indicator.append(0)
            cards.append(cardinality[cluster_id])
        feature = data_test[query_id]
        test_features.append(feature)
        test_distances.append(distances2centroids)
        test_thresholds.append([threshold+slot])
        test_targets.append(indicator)
        test_cards.append(cards)

In [5]:
cnt = 0
for i in test_cards:
    if np.sum(i) > 0:
        cnt += 1
print (cnt)

63080


In [5]:
import torch
import torch.utils.data

batch_size = 128
test_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(torch.FloatTensor(test_features), torch.FloatTensor(test_thresholds), torch.FloatTensor(test_distances), torch.FloatTensor(test_targets), torch.FloatTensor(test_cards)), batch_size=batch_size, shuffle=False)


## Global

In [6]:
from __future__ import print_function
import argparse
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

input_dimension = 200
cluster_dimension = 100
hidden_num = 256
output_num = 100

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.nn1 = nn.Linear(input_dimension, hidden_num)
        self.nn2 = nn.Linear(hidden_num, hidden_num)
#         self.nn3 = nn.Linear(hidden_num, hidden_num)
        
        self.dist1 = nn.Linear(cluster_dimension, hidden_num)
        self.dist2 = nn.Linear(hidden_num, hidden_num)
        
        self.nn4 = nn.Linear(hidden_num, hidden_num)
        self.nn5 = nn.Linear(hidden_num, output_num)
        
        self.thres1 = nn.Linear(1, hidden_num)
        self.thres2 = nn.Linear(hidden_num, 1)

    def forward(self, x, distances, thresholds):
        out1 = F.relu(self.nn1(x))
        out2 = F.relu(self.nn2(out1))
#         out3 = F.relu(self.nn3(out2))
#         print (distances.shape)
        distance1 = F.relu(self.dist1(distances))
        distance2 = F.relu(self.dist2(distance1))
        
        thresholds_1 = F.relu(self.thres1(thresholds))
        thresholds_2 = self.thres2(thresholds_1)

        out4 = F.relu(self.nn4((out2 + distance2) / 2))
        out5 = self.nn5(out2)
        
        probability = F.sigmoid(out5 + thresholds_2)
        return probability

def loss_fn(estimates, targets, cards):
    punish_idx = (estimates < 0.5).float()
    return F.mse_loss(estimates, targets) + 0.02 * torch.log(((0.5 - estimates) * cards * punish_idx).mean() + 1.0)

def print_loss(estimates, targets, cards):
    true_positive = 0.0
    true_negative = 0.0
    false_positive = 0.0
    false_negative = 0.0
    num_elements = estimates.shape[1]
    for est, tar in zip(estimates, targets):
        for i in range(num_elements):
            if est[i] < 0.5 and tar[i] == 0:
                true_negative += 1
            elif est[i] < 0.5 and tar[i] == 1:
                false_negative += 1
            elif est[i] >= 0.5 and tar[i] == 0:
                false_positive += 1
            else:
                true_positive += 1
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    total_card = cards.sum(dim=1)
#     print ('total_card: ', total_card.shape)
    miss_card = torch.FloatTensor([cards[i][((estimates[i] < 0.5).nonzero())].sum() for i in range(cards.shape[0])])
#     print ('miss_card: ', miss_card.shape)
    miss_rate = (miss_card / (total_card + 0.1)).mean()
    return precision, recall, miss_rate

## Local

In [7]:
from __future__ import print_function
import argparse
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

queries_dimension = 200
hidden_num_2 = 128

class Threshold_Model(nn.Module):
    
    def __init__(self):
        super(Threshold_Model, self).__init__()
        self.fc1 = nn.Linear(1, hidden_num_2)
        self.fc2 = nn.Linear(hidden_num_2, 1)
    
    def forward(self, threshold):
        t1 = F.relu(self.fc1(threshold))
        t2 = self.fc2(t1)
        return t2

class CNN_Model(nn.Module):
    
    def __init__(self, in_channel, out_channel, kernel_size, stride, padding, pool_type, pool_size):
        super(CNN_Model, self).__init__()
        if pool_type == 0:
            pool_layer = nn.MaxPool1d(kernel_size=pool_size, stride=pool_size)
        elif pool_type == 1:
            pool_layer = nn.AvgPool1d(kernel_size=pool_size, stride=pool_size)
        else:
            print ('CNN_Model Init Error, invalid pool_type {}'.format(pool_type))
            return
        self.layer = nn.Sequential(
            nn.Conv1d(in_channel, out_channel, kernel_size=kernel_size, stride=stride, padding=padding), 
            nn.BatchNorm1d(out_channel),
            nn.ReLU(),
            pool_layer)
        
    def forward(self, inputs):
        hid = self.layer(inputs)
        return hid

class Output_Model(nn.Module):
    
    def __init__(self, inputs_dim):
        super(Output_Model, self).__init__()
        self.fc1 = nn.Linear(inputs_dim, hidden_num_2)
        self.fc2 = nn.Linear(hidden_num_2, 1)
        
    def forward(self, queries, threshold):
        out1 = F.relu(self.fc1(queries))
        out2 = out1 + threshold
        out3 = self.fc2(out2)
        return out3

# class Model(nn.Module):
    
#     def __init__(self):
#         super(Model, self).__init__()
#         self.nn1 = nn.Linear(queries_dimension+1, hidden_num)
#         self.n1 = nn.Linear(hidden_num, hidden_num)
#         self.n2 = nn.Linear(hidden_num, hidden_num)
# #         self.n3 = nn.Linear(hidden_num, hidden_num)
# #         self.n4 = nn.Linear(hidden_num, hidden_num)
#         self.nn2 = nn.Linear(hidden_num, 1)
        
#     def forward(self, queries, threshold):
#         out1 = F.relu(self.nn1(torch.cat([queries, threshold],1)))
#         hid = out1
#         hid = F.relu(self.n1(hid))
#         hid = F.relu(self.n2(hid))
# #         hid = F.relu(self.n3(hid))
# #         hid = F.relu(self.n4(hid))
# #         hid = self.norm2(hid)
#         out2 = self.nn2(hid)
#         return out2

def loss_fn(estimates, targets, mini, maxi):
    est = unnormalize(estimates, mini, maxi)
    print (torch.cat((est, targets), 1))
    return F.mse_loss(est, targets)

def l1_loss(estimates, targets, eps=1e-5):
    return F.smooth_l1_loss(estimates, torch.log(targets))

def mse_loss(estimates, targets, eps=1e-5):
    return F.mse_loss(estimates, torch.log(targets))

def qerror_loss(preds, targets, mini, maxi):
    qerror = []
    preds = unnormal1ize_label(preds, mini, maxi)
    for i in range(len(targets)):
        if (preds[i] > targets[i]).cpu().data.numpy()[0]:
            qerror.append(preds[i]/targets[i])
        else:
            qerror.append(targets[i]/(preds[i] + 0.1))
    return torch.mean(torch.cat(qerror) ** 2)

def print_loss(estimates, targets):
    esti = torch.exp(estimates)
    qerror = []
    for i in range(esti.shape[0]):
        if esti[i] > targets[i] + 0.1:
            qerror.append((esti[i] / (targets[i] + 0.1)).item())
        else:
            qerror.append(((targets[i] + 0.1) / esti[i]).item())
    
    return F.mse_loss(esti, targets), np.mean(qerror), np.max(qerror)

In [8]:
class TunableParameters():
    
    def __init__(self, out_channel, kernel_size, stride, padding, pool_size, pool_type):
        self.out_channel = out_channel
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.pool_size = pool_size
        self.pool_type = pool_type
        
    def __repr__(self):
        return str(self.out_channel) +' '+ str(self.kernel_size) +' '+ str(self.stride) +' '+ str(self.padding) +' '+ str(self.pool_size) +' '+ str(self.pool_type)
 
    def __str__(self):
        return str(self.out_channel) +' '+ str(self.kernel_size) +' '+ str(self.stride) +' '+ str(self.padding) +' '+ str(self.pool_size) +' '+ str(self.pool_type)

In [9]:
import pickle
hyper_parameterss = []
with open('/home/sunji/ANN/glove_200_angular/saved_models/cnn_hyper_parameters.hyperpara', 'r') as handle:
    for paras in handle.readlines():
        hyper_parameters = []
        for para in paras.split(';'):
            para = para.split(' ')
            hyper_parameters.append(TunableParameters(int(para[0]), int(para[1]), int(para[2]),
                                                      int(para[3]), int(para[4]), int(para[5])))
        hyper_parameterss.append(hyper_parameters)
            

In [10]:
cnn_modelss = []
threshold_models = []
output_models = []
for idx in range(100):
    states = torch.load('/home/sunji/ANN/glove_200_angular/saved_models/local_glove_200_angular_cluster_' + str(idx) + '.model')
    hyper_para = hyper_parameterss[idx]
    cnn_models = []
    weights = [None for _ in range(len(hyper_para))]
    for key, value in states.items():
        if key != 'threshold_model_state_dict' and key != 'output_model_state_dict':
#             print (key)
            layer_id = int(key.split('_')[-1])
#             print (layer_id)
            weights[layer_id] = value
    in_channel = 1
    in_size = queries_dimension
    for weight_idx, weight in enumerate(weights):
        hyper = hyper_para[weight_idx]
        cnn_model = CNN_Model(in_channel, hyper.out_channel, hyper.kernel_size,
                              hyper.stride, hyper.padding, hyper.pool_type, hyper.pool_size)
        in_size = int((int((in_size - hyper.kernel_size + 2*(hyper.padding)) / hyper.stride) + 1) / hyper.pool_size)
        in_channel = hyper.out_channel
        cnn_model.load_state_dict(weight)
        cnn_model.eval()
        cnn_models.append(cnn_model)
    cnn_modelss.append(cnn_models)
        
    threshold_model_state_dict = states['threshold_model_state_dict']
    threshold_model = Threshold_Model()
    threshold_model.load_state_dict(threshold_model_state_dict)
    threshold_model.eval()
    threshold_models.append(threshold_model)
    
    output_model_state_dict = states['output_model_state_dict']
    output_model = Output_Model(in_size * in_channel)
    output_model.load_state_dict(output_model_state_dict)
    output_model.eval()
    output_models.append(output_model)
    

In [16]:
len(output_models)

100

In [15]:
estimatess = []
for idx in range(100):
    estimates = only_test(cnn_modelss[idx], threshold_models[idx], output_models[idx], test[idx])
    estimatess.append(estimates)

NameError: name 'only_test' is not defined

In [70]:
len(estimatess[3])

19759

## End-to-End Testing

In [11]:
def get_local_cardinality(cnn_models, threshold_model, output_model, queries, thresholds):
    queries = queries.unsqueeze(2).permute(0,2,1)
    for model in cnn_models:
        queries = model(queries)
    threshold = threshold_model(thresholds)
    queries = queries.view(queries.shape[0], -1)
    estimates = output_model(queries, threshold)
    esti = torch.exp(estimates)
    return esti.detach()

In [12]:
def print_qerror(estimates, targets):
    qerror = []
    for i in range(estimates.shape[0]):
        left = estimates[i] + 1
        right = targets[i] + 1
        if left > right:
            qerror.append((left / right).item())
        else:
            qerror.append((right / left).item())
    return qerror

In [47]:

for cluster_id in range(100):
    print (cluster_id)
    total = 0.0
    for batch_idx, (features, thresholds, distances, targets, cards) in enumerate(test_loader):
        estimates = get_local_cardinality(cnn_modelss[cluster_id], threshold_models[cluster_id],
                                             output_models[cluster_id], features, thresholds)
        print (estimates.shape, cards[:,cluster_id].shape)
        print (torch.cat((estimates, cards[:,cluster_id].unsqueeze(1)), axis=1))
        errors = print_qerror(estimates, cards[:,cluster_id])
        total += np.mean(errors)
        print (np.mean(errors))
    print (total / len(test_loader))

0
torch.Size([128, 1]) torch.Size([128])
tensor([[1436.7422,    0.0000],
        [1453.5443,    0.0000],
        [1470.5443,    0.0000],
        [1487.7424,    0.0000],
        [1505.1403,    0.0000],
        [1522.7437,   18.0000],
        [1540.5530,  287.0000],
        [1558.5698,  863.0000],
        [1576.7966, 1453.0000],
        [1595.2366, 1838.0000],
        [1613.8929, 2020.0000],
        [1632.7675, 2091.0000],
        [1651.8621, 2121.0000],
        [1671.1814, 2129.0000],
        [1690.7268, 2133.0000],
        [1710.4991, 2135.0000],
        [1730.5035, 2135.0000],
        [1750.7427, 2135.0000],
        [1771.2169, 2135.0000],
        [1791.9314, 2135.0000],
        [1812.8899, 2135.0000],
        [1834.0883, 2135.0000],
        [1855.5397, 2135.0000],
        [1877.2404, 2135.0000],
        [1899.1938, 2135.0000],
        [1921.4060, 2135.0000],
        [1943.8760, 2135.0000],
        [1966.6107, 2135.0000],
        [1989.6094, 2135.0000],
        [2012.8779, 2135.0000],

torch.Size([128, 1]) torch.Size([128])
tensor([[2.0364e+03, 2.1350e+03],
        [2.0602e+03, 2.1350e+03],
        [2.0843e+03, 2.1350e+03],
        [2.1087e+03, 2.1350e+03],
        [2.1334e+03, 2.1350e+03],
        [2.1583e+03, 2.1350e+03],
        [2.1836e+03, 2.1350e+03],
        [2.2091e+03, 2.1350e+03],
        [2.2349e+03, 2.1350e+03],
        [2.2611e+03, 2.1350e+03],
        [2.2875e+03, 2.1350e+03],
        [2.3143e+03, 2.1350e+03],
        [2.3413e+03, 2.1350e+03],
        [2.3687e+03, 2.1350e+03],
        [2.3964e+03, 2.1350e+03],
        [2.4244e+03, 2.1350e+03],
        [2.4528e+03, 2.1350e+03],
        [2.4815e+03, 2.1350e+03],
        [2.5105e+03, 2.1350e+03],
        [2.5399e+03, 2.1350e+03],
        [1.4367e+03, 0.0000e+00],
        [1.4535e+03, 0.0000e+00],
        [1.4705e+03, 0.0000e+00],
        [1.4877e+03, 0.0000e+00],
        [1.5051e+03, 0.0000e+00],
        [1.5227e+03, 2.0000e+00],
        [1.5406e+03, 1.2300e+02],
        [1.5586e+03, 6.3600e+02],
        [

tensor([[1943.8760, 2135.0000],
        [1966.6107, 2135.0000],
        [1989.6094, 2135.0000],
        [2012.8779, 2135.0000],
        [2036.4177, 2135.0000],
        [2060.2336, 2135.0000],
        [2084.3284, 2135.0000],
        [2108.7065, 2135.0000],
        [2133.3660, 2135.0000],
        [2158.3179, 2135.0000],
        [2183.5574, 2135.0000],
        [2209.0955, 2135.0000],
        [2234.9297, 2135.0000],
        [2261.0674, 2135.0000],
        [2287.5095, 2135.0000],
        [2314.2632, 2135.0000],
        [2341.3276, 2135.0000],
        [2368.7107, 2135.0000],
        [2396.4128, 2135.0000],
        [2424.4380, 2135.0000],
        [2452.7908, 2135.0000],
        [2481.4785, 2135.0000],
        [2510.4995, 2135.0000],
        [2539.8589, 2135.0000],
        [1436.7422,    0.0000],
        [1453.5443,    0.0000],
        [1470.5443,    0.0000],
        [1487.7424,    0.0000],
        [1505.1403,    0.0000],
        [1522.7437,    0.0000],
        [1540.5530,   21.0000],
        

tensor([[1855.5397, 2135.0000],
        [1877.2404, 2135.0000],
        [1899.1938, 2135.0000],
        [1921.4060, 2135.0000],
        [1943.8760, 2135.0000],
        [1966.6107, 2135.0000],
        [1989.6094, 2135.0000],
        [2012.8779, 2135.0000],
        [2036.4177, 2135.0000],
        [2060.2336, 2135.0000],
        [2084.3284, 2135.0000],
        [2108.7065, 2135.0000],
        [2133.3660, 2135.0000],
        [2158.3179, 2135.0000],
        [2183.5574, 2135.0000],
        [2209.0955, 2135.0000],
        [2234.9297, 2135.0000],
        [2261.0674, 2135.0000],
        [2287.5095, 2135.0000],
        [2314.2632, 2135.0000],
        [2341.3276, 2135.0000],
        [2368.7107, 2135.0000],
        [2396.4128, 2135.0000],
        [2424.4380, 2135.0000],
        [2452.7908, 2135.0000],
        [2481.4785, 2135.0000],
        [2510.4995, 2135.0000],
        [2539.8589, 2135.0000],
        [1436.7422,    0.0000],
        [1453.5443,    0.0000],
        [1470.5443,    0.0000],
        

131.27259549777955
torch.Size([128, 1]) torch.Size([128])
tensor([[2.4528e+03, 2.1350e+03],
        [2.4815e+03, 2.1350e+03],
        [2.5105e+03, 2.1350e+03],
        [2.5399e+03, 2.1350e+03],
        [1.4367e+03, 0.0000e+00],
        [1.4535e+03, 0.0000e+00],
        [1.4705e+03, 0.0000e+00],
        [1.4877e+03, 0.0000e+00],
        [1.5051e+03, 0.0000e+00],
        [1.5227e+03, 8.0000e+00],
        [1.5406e+03, 2.1800e+02],
        [1.5586e+03, 8.7100e+02],
        [1.5768e+03, 1.4900e+03],
        [1.5952e+03, 1.8670e+03],
        [1.6139e+03, 2.0400e+03],
        [1.6328e+03, 2.1030e+03],
        [1.6519e+03, 2.1260e+03],
        [1.6712e+03, 2.1310e+03],
        [1.6907e+03, 2.1350e+03],
        [1.7105e+03, 2.1350e+03],
        [1.7305e+03, 2.1350e+03],
        [1.7507e+03, 2.1350e+03],
        [1.7712e+03, 2.1350e+03],
        [1.7919e+03, 2.1350e+03],
        [1.8129e+03, 2.1350e+03],
        [1.8341e+03, 2.1350e+03],
        [1.8555e+03, 2.1350e+03],
        [1.8772e+03, 2.1

...

1976.2814905708656
torch.Size([128, 1]) torch.Size([128])
tensor([[3.5403e+04, 3.0646e+04],
        [3.5620e+04, 3.0646e+04],
        [3.5839e+04, 3.0646e+04],
        [3.6060e+04, 3.0646e+04],
        [3.6281e+04, 3.0646e+04],
        [3.6504e+04, 3.0646e+04],
        [3.6728e+04, 3.0646e+04],
        [3.6954e+04, 3.0646e+04],
        [3.7181e+04, 3.0646e+04],
        [3.7410e+04, 3.0646e+04],
        [3.7640e+04, 3.0646e+04],
        [3.7871e+04, 3.0646e+04],
        [3.8104e+04, 3.0646e+04],
        [3.8338e+04, 3.0646e+04],
        [3.8573e+04, 3.0646e+04],
        [3.8810e+04, 3.0646e+04],
        [3.9049e+04, 3.0646e+04],
        [3.9289e+04, 3.0646e+04],
        [3.9530e+04, 3.0646e+04],
        [3.9773e+04, 3.0646e+04],
        [2.9459e+04, 0.0000e+00],
        [2.9640e+04, 0.0000e+00],
        [2.9822e+04, 0.0000e+00],
        [3.0005e+04, 0.0000e+00],
        [3.0190e+04, 2.4480e+03],
        [3.0375e+04, 2.4190e+04],
        [3.0562e+04, 3.0304e+04],
        [3.0750e+04, 3.0

2136.8225640365854
torch.Size([128, 1]) torch.Size([128])
tensor([[3.7181e+04, 3.0646e+04],
        [3.7410e+04, 3.0646e+04],
        [3.7640e+04, 3.0646e+04],
        [3.7871e+04, 3.0646e+04],
        [3.8104e+04, 3.0646e+04],
        [3.8338e+04, 3.0646e+04],
        [3.8573e+04, 3.0646e+04],
        [3.8810e+04, 3.0646e+04],
        [3.9049e+04, 3.0646e+04],
        [3.9289e+04, 3.0646e+04],
        [3.9530e+04, 3.0646e+04],
        [3.9773e+04, 3.0646e+04],
        [2.9459e+04, 0.0000e+00],
        [2.9640e+04, 0.0000e+00],
        [2.9822e+04, 0.0000e+00],
        [3.0005e+04, 0.0000e+00],
        [3.0190e+04, 1.7000e+01],
        [3.0375e+04, 6.8710e+03],
        [3.0562e+04, 2.5653e+04],
        [3.0750e+04, 3.0364e+04],
        [3.0939e+04, 3.0625e+04],
        [3.1129e+04, 3.0641e+04],
        [3.1320e+04, 3.0646e+04],
        [3.1513e+04, 3.0646e+04],
        [3.1706e+04, 3.0646e+04],
        [3.1901e+04, 3.0646e+04],
        [3.2097e+04, 3.0646e+04],
        [3.2294e+04, 3.0

tensor([[3.5432e+04, 3.0646e+04],
        [3.5650e+04, 3.0646e+04],
        [3.5869e+04, 3.0646e+04],
        [3.6089e+04, 3.0646e+04],
        [3.6311e+04, 3.0646e+04],
        [3.6534e+04, 3.0646e+04],
        [3.6759e+04, 3.0646e+04],
        [3.6984e+04, 3.0646e+04],
        [3.7212e+04, 3.0646e+04],
        [3.7440e+04, 3.0646e+04],
        [3.7670e+04, 3.0646e+04],
        [3.7902e+04, 3.0646e+04],
        [3.8135e+04, 3.0646e+04],
        [3.8369e+04, 3.0646e+04],
        [3.8605e+04, 3.0646e+04],
        [3.8842e+04, 3.0646e+04],
        [3.9081e+04, 3.0646e+04],
        [3.9321e+04, 3.0646e+04],
        [3.9563e+04, 3.0646e+04],
        [3.9806e+04, 3.0646e+04],
        [2.9460e+04, 0.0000e+00],
        [2.9641e+04, 0.0000e+00],
        [2.9823e+04, 0.0000e+00],
        [3.0006e+04, 0.0000e+00],
        [3.0191e+04, 2.3010e+03],
        [3.0376e+04, 2.1578e+04],
        [3.0563e+04, 3.0006e+04],
        [3.0751e+04, 3.0613e+04],
        [3.0940e+04, 3.0642e+04],
        [3.113

3048.7740502655506
torch.Size([128, 1]) torch.Size([128])
tensor([[34971.6719, 30646.0000],
        [35186.6133, 30646.0000],
        [35402.8438, 30646.0000],
        [35620.3984, 30646.0000],
        [35839.2617, 30646.0000],
        [36059.5000, 30646.0000],
        [36281.1289, 30646.0000],
        [36504.1172, 30646.0000],
        [36728.4102, 30646.0000],
        [36954.1484, 30646.0000],
        [37181.2031, 30646.0000],
        [37409.6875, 30646.0000],
        [37639.6172, 30646.0000],
        [37870.9180, 30646.0000],
        [38103.6445, 30646.0000],
        [38337.8359, 30646.0000],
        [38573.4297, 30646.0000],
        [38810.4688, 30646.0000],
        [39048.9688, 30646.0000],
        [39288.9336, 30646.0000],
        [39530.3359, 30646.0000],
        [39773.2930, 30646.0000],
        [29458.9316,     0.0000],
        [29639.9336,     0.0000],
        [29822.0781,     0.0000],
        [30005.3398,     0.0000],
        [30189.7578,     0.0000],
        [30375.3105,  20

KeyboardInterrupt: 

In [13]:
model = Model()
model.load_state_dict(torch.load('/home/sunji/ANN/glove_200_angular/saved_models/global_glove_200_angular_punish_query_threshold_monotonic.model'))
model.eval()
test_loss = 0.0
precision = 0.0
recall = 0.0
miss_rate = 0.0
estimatesss = []
q_errors = []
for batch_idx, (features, thresholds, distances, targets, cards) in enumerate(test_loader):
    if batch_idx % 100 == 0:
        print (batch_idx)
    estimates = model(features, distances, thresholds)
    global_indicator = (estimates >= 0.5).float()
    local_estimates = []
    for cluster_id in range(100):
        local_estimates.append(get_local_cardinality(cnn_modelss[cluster_id], threshold_models[cluster_id],
                                                     output_models[cluster_id], features, thresholds))
    localss = torch.cat(local_estimates, dim = 1)
#     print (localss.shape, global_indicator.shape)
    cards_estimates = (localss * global_indicator).sum(dim=1).unsqueeze(1)
    cards = cards.sum(dim=1).unsqueeze(1)
#     print (cards_estimates.shape)
#     print (cards.shape)
#     print (torch.cat((cards_estimates, cards), dim=1))
    q_errors += print_qerror(cards_estimates, cards)
#     print (q_errors)
mean = np.mean(q_errors)
percent90 = np.percentile(q_errors, 90)
percent95 = np.percentile(q_errors, 95)
percent99 = np.percentile(q_errors, 99)
median = np.median(q_errors)
maxi = np.max(q_errors)
print ('Testing: Mean Error {}, Median Error {}, 90 Percent {}, 95 Percent {}, 99 Percent {}, Max Percent {}'
       .format(mean, median, percent90, percent95, percent99, maxi))
    
    
        
    

0


/home/sunji/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


100
200
300
400
500
600
700
Testing: Mean Error 2.163499464774132, Median Error 1.8379133343696594, 90 Percent 3.386749053001404, 95 Percent 4.131811189651485, 99 Percent 7.206766657829282, Max Percent 66.90528869628906
